<a href="https://colab.research.google.com/github/Sakshamarora2002/SKIN-BURN-DETECTION/blob/main/burnclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir models/
!cp -r '/content/drive/My Drive/Skin Burns.v3i.tensorflow' '/models1/'

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import seaborn as sns
from PIL import Image

np.random.seed(42)
from sklearn.model_selection import train_test_split
from scipy import stats
from sklearn.preprocessing import LabelEncoder
burn_df = pd.read_csv('/content/drive/MyDrive/Skin Burns.v3i.tensorflow/test/_annotations.csv')
le = LabelEncoder()
le.fit(burn_df['class'])
LabelEncoder()
print(list(le.classes_))

[0, 1, 2]


In [ ]:
burn_df['label'] = le.transform(burn_df["class"])
print(burn_df.sample(10))

                                              filename  width  height  class  \
198  img33_jpg.rf.7fe317f6edf890d12f7bab38b70a8a51.jpg    640     640      0   
349  img773_jpg.rf.e141b3e74219d9ab41b94f318dd23fda...    640     640      1   
33   img205_jpg.rf.16448b80307d561d51a60acb603e2710...    640     640      1   
208  img1422_jpg.rf.85342270e9e208fb24d9ef18872ba00...    640     640      0   
93   img589_jpg.rf.417bbc47c3943c33ac2731008d51e104...    640     640      2   
84   img895_jpg.rf.3df0bc0af44c79fe4a5c00e00d6a27be...    640     640      1   
374  img331_jpg.rf.f45189e8873a3adcbe9e973a32c1636c...    640     640      0   
94   img532_jpg.rf.44f4971baf4d241b0cad5635017a708a...    640     640      0   
222  img1119_jpg.rf.8e3f0dbb2b7cb831928ffdb2094d259...    640     640      1   
126  img155_jpg.rf.56a75df1507d5d131dab7058ed3032c0...    640     640      1   

     xmin  ymin  xmax  ymax  label  
198     0    25   640   640      0  
349    26     0   640   640      1  
33      

In [ ]:
from sklearn.utils import resample
print(burn_df['label'].value_counts())

1    195
0    154
2     50
Name: label, dtype: int64


In [ ]:
df_0 = burn_df[burn_df['label'] == 0]
df_1 = burn_df[burn_df['label'] == 1]
df_2 = burn_df[burn_df['label'] == 2]


In [ ]:
n_samples=500
df_0_balanced = resample(df_0, replace=True, n_samples=n_samples, random_state=42)
df_1_balanced = resample(df_1, replace=True, n_samples=n_samples, random_state=42)
df_2_balanced = resample(df_2, replace=True, n_samples=n_samples, random_state=42)

In [ ]:
burn_df_balanced = pd.concat([df_0_balanced, df_1_balanced,
                              df_2_balanced])

In [ ]:
print(burn_df_balanced['label'].value_counts())

0    500
1    500
2    500
Name: label, dtype: int64


In [ ]:
burn_df_balanced

,filename,width,height,class,xmin,ymin,xmax,ymax,label
253,img1173_jpg.rf.a4e9f0db18fea2fc58adb3e3e3e6452...,640,640,0,0,6,640,640,0
229,img1078_jpg.rf.96beaa4dbd779082dade8b09784b068...,640,640,0,5,7,640,640,0
36,img209_jpg.rf.18660b63714bac6354c0d788bac2a94a...,640,640,0,6,0,640,640,0
257,img429_jpg.rf.aad1505986cf0ec08e01dc18e1ea9762...,640,640,0,0,11,640,640,0
167,img72_jpg.rf.72bc432b736295e6b9fde0202c0f890b.jpg,640,640,0,7,0,640,640,0
...,...,...,...,...,...,...,...,...,...
38,img990_jpg.rf.17e26421bfcf0c86513322b92d5ab5cf...,640,640,2,0,0,640,640,2
87,img1067_jpg.rf.3ea6e03b470c275a5b859f29d16ded6...,640,640,2,9,0,640,640,2
131,img1007_jpg.rf.5b9cfb80593a82b8855b7ffb8363111...,640,640,2,3,6,640,640,2
218,img1245_jpg.rf.8bcab170a7fa65273e401a1e03fbcff...,640,640,2,7,8,640,640,2


In [ ]:
import pandas as pd
import os
import shutil
import tensorflow as tf

In [ ]:
burn_df = pd.read_csv('/content/drive/MyDrive/Skin Burns.v3i.tensorflow/valid/_annotations.csv')
le = LabelEncoder()
le.fit(burn_df['class'])
LabelEncoder()
print(list(le.classes_))


[0, 1, 2]


In [ ]:
#without splitting images into classes
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255., # rescaling
                                   rotation_range = 40,  # for augmentation
                                   width_shift_range = 0.2,
                                   validation_split = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1.0/255.,validation_split = 0.2)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1.0/255.)

train_generator = train_datagen.flow_from_directory("/content/drive/MyDrive/Skin Burns.v3i.tensorflow/train",
                                                    batch_size = 32,
                                                    subset="training",
                                                    class_mode = 'categorical',
                                                    target_size = (224, 224))

test_generator = test_datagen.flow_from_directory("/content/drive/MyDrive/Skin Burns.v3i.tensorflow/test",
                                                    batch_size = 32,
                                                    class_mode ='categorical',
                                                    target_size = (224, 224))

val_generator = val_datagen.flow_from_directory("/content/drive/MyDrive/Skin Burns.v3i.tensorflow/valid",
                                                subset = "validation",
                                                    batch_size = 32,
                                                    class_mode = 'categorical',
                                                    target_size = (224, 224))

Found 3332 images belonging to 3 classes.
Found 0 images belonging to 1 classes.
Found 79 images belonging to 3 classes.


In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Skin Burns.v3i.tensorflow/train/_annotations.csv')


In [ ]:
import os

base_directory = '/content/drive/MyDrive/Skin Burns.v3i.tensorflow/train/'

for index, row in df.iterrows():
    image_filename = row['filename']
    label = row['class']

    label_directory = os.path.join(base_directory, str(label))
    os.makedirs(label_directory, exist_ok=True)


In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Skin Burns.v3i.tensorflow/valid/_annotations.csv')


In [ ]:
import os


base_directory = '/content/drive/MyDrive/Skin Burns.v3i.tensorflow/test/'


for index, row in df.iterrows():
    image_filename = row['filename']
    label = row['class']

    label_directory = os.path.join(base_directory, str(label))
    os.makedirs(label_directory, exist_ok=True)


In [ ]:
train_ds=tf.keras.utils.image_dataset_from_directory(
    directory='/content/drive/MyDrive/Skin Burns.v3i.tensorflow/train',
    labels='inferred',
    label_mode='int',
    class_names=None,
    color_mode='rgb',
    batch_size=32,
    image_size=(256,256)
)
valid_ds=tf.keras.utils.image_dataset_from_directory(
    directory='/content/drive/MyDrive/Skin Burns.v3i.tensorflow/valid',
    labels='inferred',
    label_mode='int',
    class_names=None,
    color_mode='rgb',
    batch_size=32,
    image_size=(256,256)
)


Found 4164 files belonging to 3 classes.
Found 399 files belonging to 3 classes.


In [ ]:
#normalise
def process(image,label):
  image=tf.cast(image/255,tf.float32)
  return image,label

train_ds=train_ds.map(process)
valid_ds=valid_ds.map(process)

In [ ]:
#how to check output of above normalisation
import tensorflow as tf

def process(image, label):
    image = tf.cast(image / 255, tf.float32)
    return image, label

train_ds = train_ds.map(process)

num_samples_to_display = 1

for image, label in train_ds.take(num_samples_to_display):
    print("Image (Normalized):")
    print(image)
    print("Label:")
    print(label)
    print()


Image (Normalized):
tf.Tensor(
[[[[2.2722029e-03 1.9377163e-03 1.6301422e-03]
   [2.9171472e-03 2.3673589e-03 1.9290658e-03]
   [2.9863515e-03 2.3019991e-03 1.7906574e-03]
   ...
   [1.9723184e-03 1.4609765e-03 1.2418301e-03]
   [1.9607844e-03 1.4609765e-03 1.2648981e-03]
   [1.9569395e-03 1.4648212e-03 1.2956555e-03]]

  [[2.2722029e-03 1.9377163e-03 1.6301422e-03]
   [2.9171472e-03 2.3673589e-03 1.9290658e-03]
   [2.9873126e-03 2.3029603e-03 1.7916186e-03]
   ...
   [1.9876971e-03 1.4763552e-03 1.2572088e-03]
   [1.9761631e-03 1.4763552e-03 1.2802768e-03]
   [1.9723184e-03 1.4801999e-03 1.3110342e-03]]

  [[2.3144945e-03 1.9800076e-03 1.6724337e-03]
   [2.9652058e-03 2.4154172e-03 1.9771243e-03]
   [3.0372932e-03 2.3529413e-03 1.8415994e-03]
   ...
   [2.0530566e-03 1.5417146e-03 1.3225683e-03]
   [2.0415224e-03 1.5417146e-03 1.3456363e-03]
   [2.0376779e-03 1.5455594e-03 1.3763937e-03]]

  ...

  [[2.5182620e-03 2.2452904e-03 1.9223376e-03]
   [3.1747404e-03 2.7018453e-03 2.2327951e

In [ ]:
#data augmentation
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255., # rescaling
                                   rotation_range = 40,  # for augmentation
                                   width_shift_range = 0.2,
                                   validation_split = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1.0/255.,validation_split = 0.2)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1.0/255.)

train_generator = train_datagen.flow_from_directory("/content/drive/MyDrive/Skin Burns.v3i.tensorflow/train",
                                                    batch_size = 32,
                                                    subset="training",
                                                    class_mode = 'categorical',
                                                    target_size = (224, 224))


val_generator = val_datagen.flow_from_directory("/content/drive/MyDrive/Skin Burns.v3i.tensorflow/valid",
                                                subset = "validation",
                                                    batch_size = 32,
                                                    class_mode = 'categorical',
                                                    target_size = (224, 224))

Found 3332 images belonging to 3 classes.
Found 79 images belonging to 3 classes.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D,Dense,Flatten,Dropout
model=Sequential()
model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(256,256,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),padding='valid'))

model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),padding='valid'))

model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 254, 254, 32)      896       
                                                                 
 batch_normalization_9 (Bat  (None, 254, 254, 32)      128       
 chNormalization)                                                
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 127, 127, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 125, 125, 64)      18496     
                                                                 
 batch_normalization_10 (Ba  (None, 125, 125, 64)      256       
 tchNormalization)                                               
                                                      

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy

# Assuming you have three classes
num_classes = 3

# Modify your model.compile statement for sparse categorical crossentropy
model.compile(
    optimizer=Adam(),
    loss=SparseCategoricalCrossentropy(),
    metrics=[SparseCategoricalAccuracy()]
)


In [ ]:
import matplotlib.pyplot as plt

history = model.fit(x_train, y_train, epochs=100, validation_data=(x_val, y_val))

# Plot training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show()
